<a href="https://colab.research.google.com/github/JacobAshoo/NNFS/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
TODO:
-Import CIFAR10 dataset
-prep data for training
-training loop



"""

'\nTODO:\n-Droput\n-Import CIFAR10 dataset\n-prep data for training\n-training loop\n\n\n\n'

In [21]:
import numpy as np
import torch
import torchvision
import pickle
from sklearn.preprocessing import OneHotEncoder

In [ ]:
class Relu:
  def __init__(self):
    pass

  def __call__(self, x):
    self.x = x
    return torch.maximum(x, torch.tensor(0.0, device=x.device))

  def backward(self, grad):
    return grad * (self.x > 0).float()

class Softmax:
  def __init__(self):
    pass

  def __call__(self, x):
    exp_x = torch.exp(x - torch.max(x, dim=1, keepdim=True).values)
    self.softmax_out = exp_x / torch.sum(exp_x, dim=1, keepdim=True)
    return self.softmax_out

  def backward(self, grad):
    batch_size, num_classes = self.softmax_out.shape
    jacobian = torch.zeros(batch_size, num_classes, num_classes, device=grad.device)
    for i in range(num_classes):
      for j in range(num_classes):
        if i == j:
          jacobian[:, i, j] = self.softmax_out[:, i] * (1 - self.softmax_out[:, i])
        else:
          jacobian[:, i, j] = -self.softmax_out[:, i] * self.softmax_out[:, j]
    return torch.bmm(jacobian, grad.unsqueeze(-1)).squeeze(-1)


In [ ]:
class CrossEntropy:
  def __init__(self):
    pass

  def __call__(self, ypred, y, l2_reg=False, l=0.01, weights=[]):
    self.l2_reg = l2_reg
    self.l = l
    self.weights = weights
    ypred = torch.clamp(ypred, min=1e-9)
    loss = -torch.mean(torch.sum(y * torch.log(ypred), dim=1))
    if l2_reg and weights:
      loss += (l / 2) * sum(torch.sum(w ** 2) for w in weights)
    return loss

  def backward(self, ypred, y):
    grad_output = -y / ypred
    if self.l2_reg and self.weights:
      for w in self.weights:
        grad_output += self.l * w
    return grad_output

In [ ]:
class Linear:
  def __init__(self, input_size, output_size, activation, device=None, dropout=1.0):
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') if device is None else device
    self.input_size = input_size
    self.output_size = output_size
    self.w = torch.randn(output_size, input_size, device=self.device, requires_grad=True) * 0.01
    self.b = torch.zeros(output_size, device=self.device, requires_grad=True)
    self.activation = activation
    self.dropout = dropout

  def __call__(self, x, training=True):
    self.x = x.to(self.device)
    self.z = torch.matmul(self.x, self.w.T) + self.b
    self.a = self.activation(self.z)


    if training and self.dropout < 1.0:
      mask = (torch.rand(self.a.shape, device=self.device) < self.dropout).float()
      self.a *= mask
      self.a /= self.dropout

    return self.a

  def backward(self, grad):
    grad = self.activation.backward(grad)
    dw = torch.matmul(grad.T, self.x)
    db = torch.sum(grad, dim=0)
    dx = torch.matmul(grad, self.w)
    return dx, dw, db

In [ ]:
class FCNN:
  def __init__(self, input_size, output_size, dropout=False):
    self.l1 = Linear(input_size, 10, Relu())
    self.l2 = Linear(10, output_size, Softmax())
    self.layers = [self.l1, self.l2]


  def forward(self, x):
    x = self.l1(x)
    x = self.l2(x)
    return x

  def __call__(self, x):
    return self.forward(x)




In [42]:
data = torchvision.datasets.CIFAR10(root="/content",download=True)

xs = []
labels = []

for i in range(1,6):
  with open(f"/content/cifar-10-batches-py/data_batch_{i}", 'rb') as f:
    dict = pickle.load(f, encoding='bytes')
    xs.append(dict[b'data'])
    labels += dict[b'labels']


labels = np.array(labels).reshape(-1,1)
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(labels)

xs = np.array(xs)

x = xs.reshape(50000, 3072) / 255

print(x.shape)
print(y.shape)

print(x[0])
print(y[0])




Files already downloaded and verified
(50000, 3072)
(50000, 10)
[0.23137255 0.16862745 0.19607843 ... 0.54901961 0.32941176 0.28235294]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [ ]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")

model = FCNN(10, 3)

x = torch.randn(1, 10, device=device) * 0.01
y = torch.tensor([[1,0,0]], device=device)
loss_function = CrossEntropy();
ypred = model(x)

print(ypred)
loss = loss_function(ypred, y)
print(loss)


Using cpu
tensor([[0.3416, 0.3423, 0.3161]], grad_fn=<MulBackward0>)
tensor(1.0741, grad_fn=<NegBackward0>)
